In [1]:
# Import necessary initial libraries
import os
import sys
import logging
import re
from datetime import timedelta
import plotly.express as px

In [2]:
# Sets the root directory of the project as the working directory
os.chdir('..')

In [3]:
# Check the current working directory
os.getcwd()

'/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms'

In [4]:
# Import the custom module
from src import merge_geojsonGrids

In [ ]:
# Reload the module to ensure any changes are reflected
import importlib
importlib.reload(merge_geojsonGrids)

<module 'src.merge_geojsonGrids' from '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/src/merge_geojsonGrids.py'>

### Merge GeoJSON files
- Esta etapa realiza a consolidação dos arquivos GeoJSON referentes a grade estatística com os resultantes de cada step, em um único arquivo de saída (GeoPackage e GeoJSON)

In [5]:
# Lista dos arquivos GeoJSON para merge (ordem importa - primeiro arquivo tem prioridade)
# Usando pathlib para tornar o código mais legível e robusto
from pathlib import Path

testes_dir = Path(os.getcwd()) / 'data/output_code1/20cells_tests'
output_code1_files = sorted([str(f) for f in testes_dir.glob('*.geojson')])
output_code1_files


['/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/data/output_code1/20cells_tests/step1_consolidado.geojson',
 '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/data/output_code1/20cells_tests/step6_consolidado.geojson']

In [6]:
# Salva os arquivos GeoJSON consolidados em um arquivo GeoPackage
output_file_name = "steps_consolidado.gpkg"
output_path = os.path.join(testes_dir, output_file_name)

merge_geojsonGrids.merge_geojson2gpkg(output_code1_files, str(output_path))

Merge concluído!
Arquivo salvo como GeoPackage em: /Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/data/output_code1/20cells_tests/steps_consolidado.gpkg
Steps processados: ['step1_consolidado', 'step6_consolidado']
CRS mantido: EPSG:4674
Total de células únicas: 20


,id,geometry,POP10,step1_consolidado_fid,step1_consolidado_edif_ensino_total_count,step1_consolidado_edif_ensino_name_count,step1_consolidado_edif_ensino_name_ratio,step1_consolidado_edif_saude_total_count,step1_consolidado_edif_saude_name_count,step1_consolidado_edif_saude_name_ratio,...,step6_consolidado_edif_metro_ferroviaria_sigmoid_rmse,step6_consolidado_edif_metro_ferroviaria_sigmoid_pct_erro,step6_consolidado_edif_metro_ferroviaria_sigmoid_a,step6_consolidado_edif_metro_ferroviaria_sigmoid_b,step6_consolidado_edif_metro_ferroviaria_sigmoid_c,step6_consolidado_edif_metro_ferroviaria_sigmoid_d,step6_consolidado_edif_metro_ferroviaria_inflexao_idx,step6_consolidado_edif_metro_ferroviaria_inflexao_data,step6_consolidado_edif_metro_ferroviaria_sigmoid_fit_overflow,step6_consolidado_edif_metro_ferroviaria_dias_desde_inflexao
0,200ME60430N90998,"MULTIPOLYGON (((-43.97936 -19.87093, -43.97942...",118,1966,1.0,1.0,100.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
1,200ME60432N90998,"MULTIPOLYGON (((-43.97744 -19.87086, -43.97750...",30,1967,2.0,2.0,100.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
2,200ME60434N90998,"MULTIPOLYGON (((-43.97552 -19.87080, -43.97558...",0,1968,1.0,1.0,100.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
3,200ME60436N90998,"MULTIPOLYGON (((-43.97360 -19.87073, -43.97367...",0,1969,2.0,2.0,100.0,1.0,1.0,100.0,...,None,None,None,None,None,None,None,None,None,None
4,200ME60438N90998,"MULTIPOLYGON (((-43.97168 -19.87067, -43.97175...",0,1970,2.0,2.0,100.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
5,200ME60430N91000,"MULTIPOLYGON (((-43.97942 -19.86913, -43.97949...",24,1971,4.0,4.0,100.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
6,200ME60432N91000,"MULTIPOLYGON (((-43.97750 -19.86907, -43.97757...",61,1972,0.0,0.0,0.0,1.0,1.0,100.0,...,None,None,None,None,None,None,None,None,None,None
7,200ME60434N91000,"MULTIPOLYGON (((-43.97558 -19.86900, -43.97565...",0,1973,0.0,0.0,0.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
8,200ME60436N91000,"MULTIPOLYGON (((-43.97367 -19.86894, -43.97373...",0,1974,0.0,0.0,0.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
9,200ME60438N91000,"MULTIPOLYGON (((-43.97175 -19.86887, -43.97181...",0,1975,0.0,0.0,0.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None


In [7]:
# Salva os arquivos GeoJSON consolidados em um único arquivo GeoJSON
output_file_name = "steps_consolidado.geojson"
output_path = os.path.join(testes_dir, output_file_name)

merge_geojsonGrids.merge_geojson2geojson(output_code1_files, str(output_path))

Merge concluído!
Arquivo salvo como GeoJSON em: /Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/data/output_code1/20cells_tests/steps_consolidado.geojson
Steps processados: ['step1_consolidado', 'step6_consolidado']
CRS mantido: EPSG:4674
Total de células únicas: 20


,id,geometry,POP10,step1_consolidado_fid,step1_consolidado_edif_ensino_total_count,step1_consolidado_edif_ensino_name_count,step1_consolidado_edif_ensino_name_ratio,step1_consolidado_edif_saude_total_count,step1_consolidado_edif_saude_name_count,step1_consolidado_edif_saude_name_ratio,...,step6_consolidado_edif_metro_ferroviaria_sigmoid_rmse,step6_consolidado_edif_metro_ferroviaria_sigmoid_pct_erro,step6_consolidado_edif_metro_ferroviaria_sigmoid_a,step6_consolidado_edif_metro_ferroviaria_sigmoid_b,step6_consolidado_edif_metro_ferroviaria_sigmoid_c,step6_consolidado_edif_metro_ferroviaria_sigmoid_d,step6_consolidado_edif_metro_ferroviaria_inflexao_idx,step6_consolidado_edif_metro_ferroviaria_inflexao_data,step6_consolidado_edif_metro_ferroviaria_sigmoid_fit_overflow,step6_consolidado_edif_metro_ferroviaria_dias_desde_inflexao
0,200ME60430N90998,"MULTIPOLYGON (((-43.97936 -19.87093, -43.97942...",118,1966,1.0,1.0,100.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
1,200ME60432N90998,"MULTIPOLYGON (((-43.97744 -19.87086, -43.97750...",30,1967,2.0,2.0,100.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
2,200ME60434N90998,"MULTIPOLYGON (((-43.97552 -19.87080, -43.97558...",0,1968,1.0,1.0,100.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
3,200ME60436N90998,"MULTIPOLYGON (((-43.97360 -19.87073, -43.97367...",0,1969,2.0,2.0,100.0,1.0,1.0,100.0,...,None,None,None,None,None,None,None,None,None,None
4,200ME60438N90998,"MULTIPOLYGON (((-43.97168 -19.87067, -43.97175...",0,1970,2.0,2.0,100.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
5,200ME60430N91000,"MULTIPOLYGON (((-43.97942 -19.86913, -43.97949...",24,1971,4.0,4.0,100.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
6,200ME60432N91000,"MULTIPOLYGON (((-43.97750 -19.86907, -43.97757...",61,1972,0.0,0.0,0.0,1.0,1.0,100.0,...,None,None,None,None,None,None,None,None,None,None
7,200ME60434N91000,"MULTIPOLYGON (((-43.97558 -19.86900, -43.97565...",0,1973,0.0,0.0,0.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
8,200ME60436N91000,"MULTIPOLYGON (((-43.97367 -19.86894, -43.97373...",0,1974,0.0,0.0,0.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
9,200ME60438N91000,"MULTIPOLYGON (((-43.97175 -19.86887, -43.97181...",0,1975,0.0,0.0,0.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None


### Determine the time metrics of each step
- Esta etapa extrai os tempos de processamento de cada lote para cada step

In [ ]:
# Leitura do Arquivo Excel
file_name = [
    "results/1_output_grid/time_metrics/time_log_OHSOME_requests.xlsx"
]

In [ ]:
# Leitura do Arquivo Excel
df_time = pd.read_excel(file_name[0], sheet_name='step1')

In [ ]:
# Exibe as primeiras linhas para validar
display(df_time.head(10))

In [ ]:
# Exibe as últimas linhas para validar
display(df_time.tail(10))

In [ ]:
# Função para extrair tempos dos lotes
def extrair_tempos(df):
    resultados = []

    for line in df.iloc[:, 0].dropna():
        match = re.search(r'Tempo do lote (\d+): (\d+\.?\d*) min (\d+\.?\d*) seg', line)
        if match:
            lote = int(match.group(1))
            minutos = float(match.group(2))
            segundos = float(match.group(3))
            total_segundos = int(minutos * 60 + segundos)
            resultados.append({
                'Lote': lote,
                'Minutos': minutos,
                'Segundos': segundos,
                'Total (s)': total_segundos,
                'Total (min)': total_segundos / 60,
                'Total (h)': total_segundos / 3600,
                'Total (dias)': total_segundos / (3600 * 24),
                'Duração (timedelta)': timedelta(seconds=total_segundos)
            })

    return pd.DataFrame(resultados)

In [ ]:
# Aplicar a função para extrair os tempos
df_tempos = extrair_tempos(df_time)
df_tempos

In [ ]:
# Cálculo do tempo total consolidado de todos os lotes
total_horas = df_tempos['Total (h)'].sum()
total_dias = df_tempos['Total (dias)'].sum()
print(f"Tempo Total (todos os lotes): {total_horas :.2f} horas")
print(f"Tempo Total (todos os lotes): {total_dias :.2f} dias")

In [ ]:
# Gráfico de barras para visualizar os tempos de processamento
fig = px.bar(
    df_tempos,
    x='Lote',
    y='Total (min)',
    hover_data={
        'Minutos': ':.1f',
        'Segundos': ':.1f',
        'Total (h)': ':.3f'
    },
    labels={'Total (min)': 'Duração (minutos)'},
    title='Tempo de Processamento por Lote (20 células) - Step 1 (name_ratio)',
)

# Cálculo do tempo total consolidado de todos os lotes
total_horas = df_tempos['Total (h)'].sum()
total_dias = df_tempos['Total (dias)'].sum()

# Adiciona anotação com a duração total
fig.add_annotation(
    xref="paper", 
    yref="paper",
    x=0.5, 
    y=-0.35,  # Ajuste a posição vertical aqui
    text=f"Duração Total: <br>({total_horas:.2f} horas / {total_dias:.2f} dias)",
    showarrow=False,
    font=dict(size=12),
    align="center"
)

# Ajustes estéticos
fig.update_traces(marker_color='royalblue')
fig.update_layout(
    title_x=0.5,
    xaxis_title='Lote',
    yaxis_title='Duração (minutos)',
    hoverlabel=dict(bgcolor="white", font_size=13),
    margin=dict(b=100)  # Aumenta a margem inferior para caber a anotação
)

# Salva o gráfico em formato HTML
fig.write_html("step1_tempo_processamento.html")

fig.show()